<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/KoGPT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install transformers
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge
!pip install konlpy

In [1]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, DataCollatorWithPadding,TrainingArguments, Trainer, GPT2Config
from datasets import Dataset, load_dataset
from rouge import Rouge
import evaluate
from konlpy.tag import Okt
import os
import numpy as np

In [2]:
model_checkpoint = "skt/kogpt2-base-v2"

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token = '<sep>', padding_side='left')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:

# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained(model_checkpoint)

# Add the new special token to the model's config
model_config.bos_token_id = tokenizer.bos_token_id
model_config.eos_token_id = tokenizer.eos_token_id
model_config.unk_token_id = tokenizer.unk_token_id
model_config.pad_token_id = tokenizer.pad_token_id
model_config.mask_token_id = tokenizer.mask_token_id
model_config.sep_token_id = tokenizer.sep_token_id  # Add this line

# Initialize the model with the updated configuration
model = GPT2LMHeadModel.from_pretrained(model_checkpoint, config=model_config)


# 모델 테스트

In [4]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the input data (input_ids) to the GPU
input_ids = input_ids.to(device)

# Move the model to the GPU (assuming your model is already initialized)
model.to(device)
gen_ids = model.generate(input_ids,
                           max_length=32,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)

gen_ids = gen_ids.to("cpu")
generated = tokenizer.decode(gen_ids[0])
generated

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분'

정상적으로 작동하는 것 확인

# 데이터셋 전처리

In [5]:
dataset = load_dataset('drive/MyDrive', data_files={
    'train': 'nikluge-sc-2023-train.jsonl',
    'validation': 'nikluge-sc-2023-dev.jsonl',
    'test': 'nikluge-sc-2023-test.jsonl'
})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
sentence1 = []
sentence3 = []

for dictionary in dataset['train']['input']:
    sentence1.append(dictionary["sentence1"])
    sentence3.append(dictionary["sentence3"])

In [7]:
for i in range(len(sentence1)):
  sentence_1 = "첫 번째 문장 : {}".format(sentence1[i])
  sentence1[i] = sentence_1
  sentence_3 = "세 번째 문장 : {} </s> 두 번째 문장 :".format(sentence3[i])
  sentence3[i] = sentence_3

In [8]:
dataset['train'] = dataset['train'].add_column('sentence1', sentence1)
dataset['train'] = dataset['train'].add_column('sentence3', sentence3)
#dataset['train'] = dataset['train'].add_column('input_sentence',input_sentence)

In [9]:
sentence1_val = []
sentence3_val = []

for dictionary in dataset['validation']['input']:
    sentence1_val.append(dictionary["sentence1"])
    sentence3_val.append(dictionary["sentence3"])

for i in range(len(sentence1_val)):
  sentence_1 = "첫 번째 문장 : {}".format(sentence1_val[i])
  sentence1_val[i] = sentence_1
  sentence_3 = "세 번째 문장 : {} 두 번째 문장 :".format(sentence3_val[i])
  sentence3_val[i] = sentence_3

dataset['validation'] = dataset['validation'].add_column('sentence1', sentence1_val)
dataset['validation'] = dataset['validation'].add_column('sentence3', sentence3_val)
#dataset['validation'] = dataset['validation'].add_column('input_sentence', input_sentence_val)

In [10]:
sentence1_test = []
sentence3_test = []

for dictionary in dataset['test']['input']:
    sentence1_test.append(dictionary["sentence1"])
    sentence3_test.append(dictionary["sentence3"])

for i in range(len(sentence1_test)):
  sentence_1 = "첫 번째 문장 : {}".format(sentence1_test[i])
  sentence1_test[i] = sentence_1
  sentence_3 = "세 번째 문장 : {} 두 번째 문장 :".format(sentence3_test[i])
  sentence3_test[i] = sentence_3

dataset['test'] = dataset['test'].add_column('sentence1', sentence1_test)
dataset['test'] = dataset['test'].add_column('sentence3', sentence3_test)
#dataset['test'] = dataset['test'].add_column('input_sentence', input_sentence_test)

In [11]:
def tokenize_function(batch):
    sentences1 = batch['sentence1']
    sentences3 = batch['sentence3']
    outputs = batch['output']

    # Initialize lists to store the tokenized inputs and labels
    model_inputs = []
    labels = []

    for s1, s3, output in zip(sentences1, sentences3, outputs):
        # Convert list elements to strings
        sentence1_str = ''.join(map(str, s1))
        sentence3_str = ''.join(map(str, s3))
        output_str = ''.join(map(str,output))

        # Tokenize the input sentences
        inputs = tokenizer(
            '</s>' + sentence1_str + '</s>' + sentence3_str,
            padding="max_length",
            max_length=64,  # Adjust this as needed
            truncation=True,

        )

        # Tokenize the output
        output = tokenizer(
            '<sep>'+output_str,
            padding="max_length",
            max_length=64,  # Adjust this as needed
            truncation=True,

        )

        # Append tokenized inputs and labels to the lists
        model_inputs.append(inputs)
        labels.append(output)

    return {
        'input_ids': [item['input_ids'] for item in model_inputs],
        'attention_mask': [item['attention_mask'] for item in model_inputs],
        'labels': [item['input_ids'] for item in labels],
    }

In [12]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 120140
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15017
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15018
    })
})

In [16]:
tokenizer.decode(tokenized_datasets['validation']['labels'][1], skip_special_tokens= False)

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><sep> 부모님 돈은 점점 바닥나기 시작했다.'

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #동적 패딩, attention_mask, input_ids 등의 길이를 똑같이 맞춰줌

# 평가 지표 작성 및 모델 파인튜닝

평가 지표에서 차이를 보이고 있는 부분들이 있어서, 두 평가 지표를 작성해보고자 한다.

In [ ]:
def evaluate(eval_preds):
    references, predictions = eval_preds

    mecab = Mecab()

    for idx, sentence_array in enumerate(references):
        tokenized = []
        for mor in mecab.morphs(sentence_array[0]):
            tokenized.append(mor)
        references[idx] = tokenized

    for idx, s in enumerate(predictions):
        tokenized = []
        for mor in mecab.morphs(s):
            tokenized.append(mor)
        predictions[idx] = tokenized

    for idx,sentence in enumerate(predictions):
      together = " ".join(sentence)
      predictions[idx] = together
    for idx,sentence in enumerate(references):
      together = " ".join(sentence)
      references[idx] = together

    # Calculate ROUGE-1 score
    rouge = Rouge()
    results = rouge.get_scores(predictions, references, avg=True)
    rouge_1_score = results['rouge-1']['f']

    #Calculate Bleu Score
    google_bleu = evaluate.load("google_bleu")
    results = google_bleu.compute(predictions=predictions, references=references)
    bleu_score = results['google_bleu']

    #Calculate BERTScore
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    bert_score_f1 = sum(results['f1'])/len(predictions)

    # Calculate the mean of the three scores
    mean_score = (rouge_1_score + bleu_score + bert_score_f1) / 3.0

    return mean_score

In [18]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    okt = Okt()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = [" ".join(okt.morphs(pred.strip())) for pred in decoded_preds]
    decoded_labels = [" ".join(okt.morphs(label.strip())) for label in decoded_labels]

    # Calculate ROUGE-1 score
    rouge = Rouge()
    results = rouge.get_scores(decoded_preds, decoded_labels, avg=True)
    rouge_1_score = results['rouge-1']['f']

    #Calculate Bleu Score
    google_bleu = evaluate.load("google_bleu")
    results = google_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = results['google_bleu']

    #Calculate BERTScore
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=decoded_preds, references=decoded_labels, model_type="distilbert-base-uncased")
    bert_score_f1 = sum(results['f1'])/len(predictions)

    # Calculate the mean of the three scores
    mean_score = (rouge_1_score + bleu_score + bert_score_f1) / 3.0

    return mean_score

In [19]:
def preprocess_logits_for_text_generation(logits, generated_token, max_length=50, temperature=0.7):
    # Apply temperature to the logits to control the randomness of text generation
    logits /= temperature

    # Reshape generated_token to have the same shape as logits
    generated_token = generated_token.unsqueeze(1)  # Add a new dimension

    # Concatenate the generated token to the input sequence
    generated_text = torch.cat([logits, generated_token], dim=1)

    # Truncate or pad the generated text to the specified max_length
    if generated_text.shape[1] > max_length:
        generated_text = generated_text[:, :max_length]
    elif generated_text.shape[1] < max_length:
        padding = torch.zeros((generated_text.shape[0], max_length - generated_text.shape[1]), dtype=torch.long)
        generated_text = torch.cat([generated_text, padding], dim=1)

    return generated_text


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
batch_size = 32
num_train_epochs = 3
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args  = TrainingArguments(
    output_dir=f"{model_name}-finetuned-koGPT-complete_story",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_train_epochs,
    #predict_with_generate=True,
    #구글링 결과는 이걸 True로 하라는데 False 로 하니까 갑자기 해결됐다. 뭐지? 아마도 이걸 True로 해서 list, 즉 결과를 반환해주어버려서 compute_metrics에 list가 들어갔나?
    #해답 찾았는데, 결국 이걸 True로 해야 compute_metrics가 작동을 하는 것임. compute_metrics가 아직도 오류를 갖고 있음을 알 수 있음.
    logging_steps = logging_steps,
    push_to_hub=True,
)

In [22]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_text_generation
)


In [23]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: ignored

In [ ]:
trainer.evaluate()

{'eval_loss': 0.9827202558517456,
 'eval_runtime': 27.0987,
 'eval_samples_per_second': 554.16,
 'eval_steps_per_second': 17.344,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub(commit_message="Training complete", tags="text generation")

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

'https://huggingface.co/Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story/tree/main/'

In [ ]:
tokenized_datasets['test']['labels'][1]

In [ ]:
predictions= trainer.predict(tokenized_datasets['test'])

AttributeError: ignored

In [ ]:
predictions[1][2]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [ ]:
generated_text = tokenizer.decode(predictions[0], skip_special_tokens=True)
print(generated_text)

이는이는 기 기 기 안내 것을 말다. 안내 낙엽 낙엽 산 낙엽 낙엽 옮겨 붙다. 옮겨벌벌 벌 벌에게냐라고 말 했다.된다고 나를 나를 나를 나를주지 않았다.주지 AT AT AT AT AT AT 은행에 갔다. 뽑 그래서 그래서 그래서 그래서 119 119에 불렀다.고를 남편의 남편의 남편의 남편을백했다.백을 그들은 그들은 밖으로 밖으로 나갔다. 나갔 그 그 그 자전거거를춰갔다.춰 나는이이이웠다.이 대체 학생들은 강의 강의 불만을 못했다. 줄어들 따라 따라 따라 움직다. 따라 세계수는 부담 부담 부담감을 못했다. 부담키 쿠를를 칭찬다고 칭찬었다. 칭찬 언 기억 기억 기억 기억 기억 말해 말했다. 말 그는 그는 그는 그는 그는 일을 일을 일을했다. 일을 나는 나는 나는난 화를 말했다. 말 부모 부모 부모 부모 부모 부모 선물을 선물을하셨다.하 그녀는 그녀는 아들을 아들을 찾아 생각했다. 수 한 한 아이들이 아이들이 책을 책을 책을기 시작다. 앉에게에게냐냐냐고 물었다.고빠빠빠빠빠빠 표를 표를다고 말했다.다고 견인 견인 견인 견인 견인 전화를 요청했다. 전화를우스를우스를우스를보니우스를 색이 색이 발견다.색이 교수 교수 교수 교수님께 털어 구했다. 털어 동 동을 화를쳤다. 들 등급 식 식 식 식당을 요구다.당을 나는 친구 친구 나 평평을평을했다.평을 숙주 숙주 숙주 숙주 숙주 숙주 꺼 먹다.고 그에게 그에게 그에게 그에게 술을 잔 건 말했다. 건 준 준 준 출장근근근 갔다.근 엄마는준 엄 엄 엄마가 씻라고라고 화를셨했다.화를 이탈당에서뉴뉴있게 주문다. 주문 만 만 만 만 꽃 꽃잎 꽃을 꽃을 꺾다. 꺾마는 엄 엄 키우는 키우 키우 키우하 말했다.라고 형 형 형 돈을 돈을 갚했다. 사업이역에서역에서 돈을 나를 놀 놀다. 놀 조카 조카 조카 조카 조카 잘다고 못다.다고 피가 피가 피가 피가렀다. 피가 주전 주전 주전 주전 주전 워다.워 그림을이는 그림을 그림을 그림을 그림을 그림을 그렸했다. 그렸 카드를 카드를 카드를증을 요구했다. 달라고 침 침 프레 프레지 물어봤다. 바꾸 짜

In [ ]:
from transformers import pipeline

hub_model_id = "Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story"
generator = pipeline("text-generation", model=hub_model_id)

In [ ]:
generator(dataset['test']['sentence1'][1] + dataset['test']['sentence3'][1], max_length=64)

[{'generated_text': '첫 번째 문장 : 언덕 위에 있는 남성이 팔을 위로 들고 있었다.세 번째 문장 : 그러자 다른 사람이 제발 생쇼하지 말라고 조소했다. 두 번째 문장 :에게에게 가서하여 먼저까지를 부딪 부딪'}]

In [ ]:
dataset['test']['sentence1'][0]

'첫 번째 문장 : 서영이는 기차에 타서 자리에 앉았다.'